In [59]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.models.mobilenetv3 import mobilenet_v3_small
from PIL import Image
import numpy as np
import math
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

In [60]:
# Load MobileNetV3 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
mobilenet_model = mobilenet_v3_small()
mobilenet_model.classifier[3] = nn.Linear(mobilenet_model.classifier[3].in_features, 4)
mobilenet_checkpoint = torch.load("D:/Projects/ML/GCN/Framework/cuda_tree_model/model_weights_v2/mobilenetv3_epoch_5.pth", map_location=device)
mobilenet_model.load_state_dict(mobilenet_checkpoint)
mobilenet_model = mobilenet_model.to(device)
mobilenet_model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [62]:
# Dummy GCN model class (customize this based on your actual implementation)
class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [63]:
# Load GCN model
gcn_model = GCNModel(in_channels=256, hidden_channels=82, out_channels=4)
gcn_checkpoint = torch.load("D:/Projects/ML/GCN/Framework/cuda_tree_model/gcn_model_v8/gcn_epoch_269.pth", map_location=device)
gcn_model.load_state_dict(gcn_checkpoint)
gcn_model = gcn_model.to(device)
gcn_model.eval()

GCNModel(
  (conv1): GCNConv(256, 82)
  (conv2): GCNConv(82, 4)
)

In [64]:
# Transform for input images (you might not need this if you're using pre-computed features)
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [65]:
# Path setup
test_annotations_dir = "D:/Projects/ML/GCN/Dataset_GCN_classifier_mobilenetv3/test"  # Directory containing test txt files
test_images_dir = "D:/Projects/ML/GCN/Dataset_classifier/test/images"  # Directory containing test images
output_annotations_dir = "D:/Projects/ML/GCN/Outputs/results_comprehensive_vX_combo"  # Output directory for predictions
os.makedirs(output_annotations_dir, exist_ok=True)

In [66]:
def read_annotations(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 204:
            label = parts[0]
            x1, y1, x2, y2 = map(int, parts[1:5])
            feature_vector = np.array(list(map(float, parts[5:])))
            annotations.append((label, x1, y1, x2, y2, feature_vector))
    return annotations

In [67]:
def classify_with_mobilenet(image_crop):
    image_tensor = transform(image_crop).unsqueeze(0).to(device)
    with torch.no_grad():
        output = mobilenet_model(image_tensor)
        sp = nn.Sigmoid()
        confidence = sp(torch.max(output[0])).cpu().numpy() / (sp(output[0][0]) + sp(output[0][1]) + sp(output[0][2]) + sp(output[0][3])).cpu().numpy()
        _, predicted_class = torch.max(output, dim=1)
    return predicted_class.item(), confidence

In [68]:
import networkx as nx

In [69]:
def label_mapping(label):
    if label.lower() == 'ld':
        return 0
    if label.lower() == 'hd':
        return 1
    if label.lower() == 'other':
        return 2
    if label.lower() == 'h':
        return 3

In [70]:
# for annotation_file in os.listdir(test_annotations_dir):
#     full_path = os.path.join(test_annotations_dir, annotation_file)
# 
#     # Read annotations
#     annotations = read_annotations(full_path)
# 
#     # Load related image
#     image_filename = annotation_file.replace('.txt', '.jpg')  # assuming images are in JPEG format
#     image_path = os.path.join(test_images_dir, image_filename)
#     # image = Image.open(image_path).convert('RGB')
#     image = cv2.imread(image_path)
# 
#     # Separate annotations for MobileNet and GCN
#     mobilenet_annotations = []
#     gcn_annotations = []
# 
#     for annotation in annotations:
#         label, x1, y1, x2, y2, feature_vector = annotation
#         # label = label_mapping(label)
#         # Crop the image to the bounding box
#         image_crop = image[y1:y2+1, x1:x2+1]
# 
#         # Classify with MobileNetV3
#         predicted_class, confidence = classify_with_mobilenet(image_crop)
# 
#         if confidence > 0.45:
#             mobilenet_annotations.append((label, x1, y1, x2, y2, predicted_class))
#         else:
#             gcn_annotations.append((label, x1, y1, x2, y2, feature_vector))
#         #mobilenet_annotations.append((label, x1, y1, x2, y2, predicted_class))
# 
#     #         
#     # if len(gcn_annotations) == 1:
#     #     label, x1, y1, x2, y2, feature_vector = gcn_annotations[0]
#     #     image_crop = image[y1:y2+1, x1:x2+1]
#     #     predicted_class, confidence = classify_with_mobilenet(image_crop)
#     #     mobilenet_annotations.append((label, x1, y1, x2, y2, predicted_class))
#     # 
#     # if len(gcn_annotations) > 1:
#     #     # Create a graph for GCN
#     #     G = nx.Graph()
#     #     node_features = []
#     # 
#     #     edges = []
#     # 
#     #     # Add nodes
#     #     for i, (label, x1, y1, x2, y2, feature_vector) in enumerate(gcn_annotations):
#     #         G.add_node(i, feature=feature_vector)
#     #         node_features.append(feature_vector)
#     # 
#     #     # Add edges - fully connected graph
#     #     for i in range(len(gcn_annotations)):
#     #         for j in range(i + 1, len(gcn_annotations)):
#     #             edges.append([i, j])
#     # 
#     #     # Extract edge index
#     #     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
#     #     node_features_tensor = torch.tensor(node_features, dtype=torch.float)
#     # 
#     #     data = Data(node_features_tensor, edge_index, node_features_tensor).to(device)
#     #     # Run GCN classifier
#     #     # gcn_features = torch.tensor(node_features, dtype=torch.float32).to(device)
#     #     with torch.no_grad():
#     #         gcn_output = gcn_model(data)
#     #         gcn_predictions = gcn_output.max(1)[1].cpu().numpy()
#     # 
#     # if (len(gcn_annotations) > 1):
#     #     # Combine results
#     #     combined_annotations = mobilenet_annotations + \
#     #                            [(label, x1, y1, x2, y2, gcn_predictions[i])
#     #                             for i, (label, x1, y1, x2, y2, _) in enumerate(gcn_annotations)]
#     # else:
#     #     combined_annotations = mobilenet_annotations
#     # 
#     # # Save output
#     # output_path = os.path.join(output_annotations_dir, annotation_file)
#     # with open(output_path, 'w') as output_file:
#     #     for label, x1, y1, x2, y2, predicted_label in combined_annotations:
#     #         output_file.write(f"{label} {x1} {y1} {x2} {y2} {predicted_label}\n")
#     #     print(annotation_file)
#     # 
#     # # Collect for overall results
#     # results.extend(combined_annotations)
# 
#         # Save output
#     output_path = os.path.join(output_annotations_dir, annotation_file)
#     with open(output_path, 'w') as output_file:
#         for label, x1, y1, x2, y2, predicted_label in mobilenet_annotations:
#             output_file.write(f"{label} {x1} {y1} {x2} {y2} {predicted_label}\n")
# 
#     # Collect for overall results
#     results.extend(mobilenet_annotations)

In [71]:
# # Calculate metrics
# true_labels = [int(label) for label, _, _, _, _, _ in results]
# predicted_labels = [predicted_label for _, _, _, _, _, predicted_label in results]
# precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
# print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

In [72]:
for steps in range(0, 105, 5):
    conf_threshold = round(float(steps / 100), 2)
    
    results = []

    for annotation_file in os.listdir(test_annotations_dir):
        full_path = os.path.join(test_annotations_dir, annotation_file)

        # Read annotations
        annotations = read_annotations(full_path)
    
        # Load related image
        image_filename = annotation_file.replace('.txt', '.jpg')  # assuming images are in JPEG format
        image_path = os.path.join(test_images_dir, image_filename)
        # image = Image.open(image_path).convert('RGB')
        image = cv2.imread(image_path)
    
        # Separate annotations for MobileNet and GCN
        mobilenet_annotations = []
        gcn_annotations = []
    
        for annotation in annotations:
            label, x1, y1, x2, y2, feature_vector = annotation
            # label = label_mapping(label)
            # Crop the image to the bounding box
            image_crop = image[y1:y2 + 1, x1:x2 + 1]
    
            # Classify with MobileNetV3
            predicted_class, confidence = classify_with_mobilenet(image_crop)
    
            if confidence > conf_threshold:
                mobilenet_annotations.append((label, x1, y1, x2, y2, predicted_class))
            else:
                # gcn_annotations.append((label, x1, y1, x2, y2, feature_vector))
                mobilenet_annotations.append((label, x1, y1, x2, y2, -1))
    

        # if len(gcn_annotations) == 1:
        #     label, x1, y1, x2, y2, feature_vector = gcn_annotations[0]
        #     image_crop = image[y1:y2+1, x1:x2+1]
        #     predicted_class, confidence = classify_with_mobilenet(image_crop)
        #     mobilenet_annotations.append((label, x1, y1, x2, y2, predicted_class))
        # 
        # if len(gcn_annotations) > 1:
        #     # Create a graph for GCN
        #     G = nx.Graph()
        #     node_features = []
        # 
        #     edges = []
        #     distances = []
        #     mean_distance=10000
        # 
        #     # Add nodes
        #     for i, (label, x1, y1, x2, y2, feature_vector) in enumerate(gcn_annotations):
        #         G.add_node(i, feature=feature_vector)
        #         node_features.append(feature_vector)
        # 
        #     for i in range(len(gcn_annotations)):
        #         for j in range(i+1, len(gcn_annotations)):
        #             x1, y1, x2, y2 = gcn_annotations[i][1:5]
        #             x1_, y1_, x2_, y2_ = gcn_annotations[j][1:5]
        #             distance = math.sqrt(((x1+x2) / 2 - (x1_+x2_) / 2)**2 + ((y1+y2) / 2 - (y1_+y2_) / 2)**2)
        #             distances.append(distance)
        #     
        #     mean_distance = sum(distances) / len(distances) + 1
        #     
        #     for i in range(len(gcn_annotations)):
        #         for j in range(i+1, len(gcn_annotations)):
        #             x1, y1, x2, y2 = gcn_annotations[i][1:5]
        #             x1_, y1_, x2_, y2_ = gcn_annotations[j][1:5]
        #             distance = math.sqrt(((x1+x2) / 2 - (x1_+x2_) / 2)**2 + ((y1+y2) / 2 - (y1_+y2_) / 2)**2)
        #             if distance < mean_distance:
        #                 edges.append([i, j])
        # 
        #     # # Add edges - fully connected graph
        #     # for i in range(len(gcn_annotations)):
        #     #     for j in range(i + 1, len(gcn_annotations)):
        #     #         edges.append([i, j])
        # 
        #     # Extract edge index
        #     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        #     node_features_tensor = torch.tensor(node_features, dtype=torch.float)
        # 
        #     data = Data(node_features_tensor, edge_index, node_features_tensor).to(device)
        #     # Run GCN classifier
        #     # gcn_features = torch.tensor(node_features, dtype=torch.float32).to(device)
        #     with torch.no_grad():
        #         gcn_output = gcn_model(data)
        #         gcn_predictions = gcn_output.max(1)[1].cpu().numpy()
        # 
        # if (len(gcn_annotations) > 1):
        #     # Combine results
        #     combined_annotations = mobilenet_annotations + \
        #                            [(label, x1, y1, x2, y2, gcn_predictions[i])
        #                             for i, (label, x1, y1, x2, y2, _) in enumerate(gcn_annotations)]
        # else:
        #     combined_annotations = mobilenet_annotations

        # Save output
        output_path = os.path.join(output_annotations_dir, annotation_file)
        with open(output_path, 'w') as output_file:
            for label, x1, y1, x2, y2, predicted_label in mobilenet_annotations:
                output_file.write(f"{label} {x1} {y1} {x2} {y2} {predicted_label}\n")

        # Collect for overall results
        results.extend(mobilenet_annotations)
    
    # Calculate metrics
    true_labels = [int(label) for label, _, _, _, _, _ in results]
    predicted_labels = [int(predicted_label) for _, _, _, _, _, predicted_label in results]
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
    res = precision_recall_fscore_support(true_labels, predicted_labels, average=None)
    p, r, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
    print(f"'0': P: {res[0][0]} , R: {res[1][0]} , F1: {res[2][0]} , conf: {conf_threshold}")
    print(f"'1': P: {res[0][1]} , R: {res[1][1]} , F1: {res[2][1]} , conf: {conf_threshold}")
    print(f"'2': P: {res[0][2]} , R: {res[1][2]} , F1: {res[2][2]} , conf: {conf_threshold}")
    print(f"'3': P: {res[0][3]} , R: {res[1][3]} , F1: {res[2][3]} , conf: {conf_threshold}")
    print(f"weighted: P: {p} , R: {r} , F1: {f1} , conf: {conf_threshold}")
    # print(f'Conf: {conf_threshold}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1 Score: {f1:.6f}')

'0': P: 0.8984836802878438 , R: 0.9479392624728851 , F1: 0.9225491489642433 , conf: 0.0
'1': P: 0.8086070215175538 , R: 0.7702265372168284 , F1: 0.7889502762430939 , conf: 0.0
'2': P: 0.9953892534137259 , R: 0.9952127659574468 , F1: 0.9953010018618672 , conf: 0.0
'3': P: 0.76 , R: 0.29457364341085274 , F1: 0.4245810055865922 , conf: 0.0
weighted: P: 0.9391479036527874 , R: 0.9415961190906497 , F1: 0.9375780193474507 , conf: 0.0
'0': P: 0.8984836802878438 , R: 0.9479392624728851 , F1: 0.9225491489642433 , conf: 0.05
'1': P: 0.8086070215175538 , R: 0.7702265372168284 , F1: 0.7889502762430939 , conf: 0.05
'2': P: 0.9953892534137259 , R: 0.9952127659574468 , F1: 0.9953010018618672 , conf: 0.05
'3': P: 0.76 , R: 0.29457364341085274 , F1: 0.4245810055865922 , conf: 0.05
weighted: P: 0.9391479036527874 , R: 0.9415961190906497 , F1: 0.9375780193474507 , conf: 0.05
'0': P: 0.8984836802878438 , R: 0.9479392624728851 , F1: 0.9225491489642433 , conf: 0.1
'1': P: 0.8086070215175538 , R: 0.770226537

D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'0': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.35
'1': P: 0.8984575835475579 , R: 0.9476681127982647 , F1: 0.9224069675376089 , conf: 0.35
'2': P: 0.808390022675737 , R: 0.7691477885652643 , F1: 0.7882808181315644 , conf: 0.35
'3': P: 0.9955642299503193 , R: 0.9948581560283688 , F1: 0.9952110677545228 , conf: 0.35
weighted: P: 0.9395941223929101 , R: 0.941215637781794 , F1: 0.9374794007602928 , conf: 0.35


D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'0': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.4
'1': P: 0.899974220159835 , R: 0.946583514099783 , F1: 0.9226906303687062 , conf: 0.4
'2': P: 0.8130733944954128 , R: 0.7648327939590076 , F1: 0.7882156753752084 , conf: 0.4
'3': P: 0.99644128113879 , R: 0.9929078014184397 , F1: 0.9946714031971581 , conf: 0.4
weighted: P: 0.9412412170932896 , R: 0.9391229905830876 , F1: 0.9370134052335982 , conf: 0.4


...

D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca

'0': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.9
'1': P: 0.9948453608247423 , R: 0.05233188720173536 , F1: 0.0994332818134982 , conf: 0.9
'2': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.9
'3': P: 1.0 , R: 0.7210992907801419 , F1: 0.8379519934068198 , conf: 0.9
weighted: P: 0.8854741454125036 , R: 0.40521259393132314 , F1: 0.48442492020761385 , conf: 0.9


D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca

'0': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.95
'1': P: 0.975609756097561 , R: 0.010845986984815618 , F1: 0.02145347278090641 , conf: 0.95
'2': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 0.95
'3': P: 1.0 , R: 0.6418439716312057 , F1: 0.7818574514038877 , conf: 0.95
weighted: P: 0.8787262228182064 , R: 0.3481403976029678 , F1: 0.4269757855544479 , conf: 0.95
'0': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 1.0
'1': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 1.0
'2': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 1.0
'3': P: 0.0 , R: 0.0 , F1: 0.0 , conf: 1.0
weighted: P: 0.0 , R: 0.0 , F1: 0.0 , conf: 1.0


D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Projects\ML\GCN\Framework\cuda_tree_model\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca